In [36]:
import requests
import pprint
pp = pprint.PrettyPrinter(indent=4)
import numpy as np
import pandas as pd
import json
import re
import os, sys

In [37]:
main_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir)

# Functions That Will Be Called

In [38]:
def prop_type(string):
    type_ = re.findall('(.+?)\(', string)[1].lower().strip(')')
    return type_

def player_name(string):
    name = re.findall('(.+?)\(', string)[0].lower().strip()
    return name

In [39]:
def get_picks(pp, pin, league):
    df = pp[pp['league']==league]
    df = df.merge(pin, how='left', on=['player_name', 'prop_type'])
    df.dropna(inplace=True)
    df.drop(columns=['prop_id', 'over_id', 'under_id', 'under_points'], inplace=True)
    df.rename(columns={'over_points':'pinnacle_line'}, inplace=True)
    
    df['favored'] = np.where(df['min_price']==df['over_price'], 'over', 'under')
    
    df['edge'] = np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='under'), 'edge',
                         np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='over'), 'edge',
                                 np.where(df['pinnacle_line']==df['pp_line'], 'equal',
                                         np.where((df['pinnacle_line']<df['pp_line'])&(df['favored']=='over'), 'buffer',
                                                 np.where((df['pinnacle_line']>df['pp_line'])&(df['favored']=='under'), 'buffer', 'else')))))
    df.sort_values(by='min_price', inplace=True)
    return df

# Prize Picks Projections Scrape

In [40]:
f = open(main_dir + "/PrizePicks/projections.json")
data = json.load(f)
pp = pd.json_normalize(data['included'])
pp1 = pp[pp['type'] == 'new_player']
pp2 = pd.json_normalize(data['data'])

pp1 = pp1.rename(columns = {'id' : 'playerid'})
pp2 = pp2.rename(columns = {'relationships.new_player.data.id' : 'playerid'})

mergedDF = pp2.merge(pp1, on = 'playerid', how = 'left')

fdf = mergedDF[['attributes.line_score','attributes.stat_type','attributes.name','attributes.league']]

In [41]:
fdf.rename(columns={'attributes.line_score':'pp_line', 'attributes.stat_type':'prop_type', 'attributes.name':'player_name', 
                   'attributes.league':'league'}, inplace=True)
fdf['player_name'] = fdf['player_name'].str.lower()
fdf['league'] = fdf['league'].str.lower()
fdf['prop_type'] = fdf['prop_type'].str.lower()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

In [42]:
# changing prop types to match pinnacle
# keep adding to this until complete

# MLB PROPS
fdf['prop_type'] = np.where(fdf['prop_type']=='earned runs allowed', 'earned runs',
                           np.where(fdf['prop_type']=='strikeouts', 'total strikeouts',
                                   fdf['prop_type']))

#NFL Props
fdf['prop_type'] = np.where(fdf['prop_type']=='pass yards', 'pass yds', 
                           np.where(fdf['prop_type']=='receiving yards', 'receiving yds', 
                                    np.where(fdf['prop_type']=='rush yards', 'rush yds',
                                            fdf['prop_type'])))

#NHL Props
fdf['prop_type'] = np.where(fdf['prop_type']=='goalie saves', 'saves', fdf['prop_type'])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

# Pinnacle NFL Scraping

In [43]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NFL/nfl_matchups.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()
#print(count)

In [44]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NFL/nfl_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()
#print(count)

In [45]:
nfl_propd = pd.DataFrame(player_props)
nfl_prices = pd.DataFrame(prop_prices)

In [46]:
nfl_df = pd.merge(nfl_propd, nfl_prices, how='left', on=['prop_id', 'over_id', 'under_id'])
nfl_df.dropna(inplace=True)
nfl_df['min_price'] = np.where(nfl_df['over_price']<=nfl_df['under_price'], 
                               nfl_df['over_price'], nfl_df['under_price'])

In [47]:
nfl_df['player_name'] = nfl_df['prop_name'].apply(player_name)
nfl_df['prop_type'] = nfl_df['prop_name'].apply(prop_type)
nfl_df.drop(columns='prop_name', inplace=True)
nfl_df.drop_duplicates(inplace=True)

In [48]:
nfl = get_picks(fdf, nfl_df, 'nfl')
nfl.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
16,2.0,receptions,tre'quan smith,nfl,2.5,128.0,-173.0,-173.0,under,buffer
13,2.0,receptions,marquez callaway,nfl,2.5,111.0,-148.0,-148.0,under,buffer
0,30.5,rush yds,kyler murray,nfl,29.5,-125.0,-106.0,-125.0,over,buffer
7,32.5,receiving yds,alvin kamara,nfl,33.5,-123.0,-108.0,-123.0,over,edge
4,65.5,rush yds,alvin kamara,nfl,60.5,-112.0,-119.0,-119.0,under,edge
9,47.5,receiving yds,zach ertz,nfl,47.5,-118.0,-112.0,-118.0,over,equal
14,48.5,receiving yds,rondale moore,nfl,48.5,-112.0,-118.0,-118.0,under,equal
15,27.5,receiving yds,tre'quan smith,nfl,27.5,-112.0,-118.0,-118.0,under,equal
18,5.5,receiving yds,keith kirkwood,nfl,5.5,-118.0,-112.0,-118.0,over,equal
8,60.5,receiving yds,chris olave,nfl,60.5,-113.0,-117.0,-117.0,under,equal


# Pinnacle MLB Scraping

In [49]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/MLB/mlb_matchups.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)

# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()

In [50]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/MLB/mlb_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()

In [51]:
mlb_propd = pd.DataFrame(player_props)
mlb_prices = pd.DataFrame(prop_prices)

In [52]:
mlb_df = pd.merge(mlb_propd, mlb_prices, how='left', on=['prop_id', 'over_id', 'under_id'])
mlb_df.dropna(inplace=True)
mlb_df['min_price'] = np.where(mlb_df['over_price']<=mlb_df['under_price'], 
                               mlb_df['over_price'], mlb_df['under_price'])

In [53]:
mlb_df['player_name'] = mlb_df['prop_name'].apply(player_name)
mlb_df['prop_type'] = mlb_df['prop_name'].apply(prop_type)

mlb_df.drop(columns='prop_name', inplace=True)
mlb_df.drop_duplicates(inplace=True)

In [54]:
mlb = get_picks(fdf, mlb_df, 'mlb')
mlb.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
10,4.0,hits allowed,blake snell,mlb,4.5,135.0,-185.0,-185.0,under,buffer
29,2.0,earned runs,justin verlander,mlb,1.5,-183.0,134.0,-183.0,over,buffer
25,4.0,total strikeouts,jameson taillon,mlb,3.5,-164.0,122.0,-164.0,over,buffer
31,6.0,total strikeouts,justin verlander,mlb,5.5,-160.0,119.0,-160.0,over,buffer
52,0.5,total bases,jose trevino,mlb,0.5,-157.0,117.0,-157.0,over,equal
47,0.5,total bases,trey mancini,mlb,0.5,-144.0,108.0,-144.0,over,equal
22,0.5,total bases,bryson stott,mlb,0.5,-143.0,107.0,-143.0,over,equal
21,0.5,total bases,ha-seong kim,mlb,0.5,-142.0,107.0,-142.0,over,equal
50,0.5,total bases,josh donaldson,mlb,0.5,-140.0,105.0,-140.0,over,equal
8,6.0,total strikeouts,blake snell,mlb,6.5,104.0,-139.0,-139.0,under,buffer


# Pinnacle NHL Scraping

In [55]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NHL/nhl_matchups.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()
#print(count)

In [56]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NHL/nhl_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()
#print(count)

In [57]:
nhl_propd = pd.DataFrame(player_props)
nhl_prices = pd.DataFrame(prop_prices)

In [58]:
nhl_df = pd.merge(nhl_propd, nhl_prices, how='left', on=['prop_id', 'over_id', 'under_id'])
nhl_df.dropna(inplace=True)
nhl_df['min_price'] = np.where(nhl_df['over_price']<=nhl_df['under_price'], 
                               nhl_df['over_price'], nhl_df['under_price'])

In [59]:
nhl_df['player_name'] = nhl_df['prop_name'].apply(player_name)
nhl_df['prop_type'] = nhl_df['prop_name'].apply(prop_type)

nhl_df.drop(columns='prop_name', inplace=True)
nhl_df.drop_duplicates(inplace=True)

In [60]:
nhl = get_picks(fdf, nhl_df, 'nhl')
nhl.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
9,2.0,shots on goal,tony deangelo,nhl,2.5,146.0,-202.0,-202.0,under,buffer
18,1.0,points,nathan mackinnon,nhl,1.5,141.0,-195.0,-195.0,under,buffer
24,3.0,shots on goal,mikko rantanen,nhl,2.5,-192.0,141.0,-192.0,over,buffer
7,2.0,shots on goal,ivan provorov,nhl,2.5,140.0,-191.0,-191.0,under,buffer
30,3.0,shots on goal,kyle connor,nhl,2.5,-190.0,139.0,-190.0,over,buffer
36,2.0,shots on goal,ryan o'reilly,nhl,1.5,-184.0,135.0,-184.0,over,buffer
25,3.0,shots on goal,valeri nichushkin,nhl,2.5,-184.0,135.0,-184.0,over,buffer
28,2.0,shots on goal,blake wheeler,nhl,2.5,135.0,-184.0,-184.0,under,buffer
4,3.5,shots on goal,matthew tkachuk,nhl,2.5,-183.0,134.0,-183.0,over,buffer
23,3.0,shots on goal,cale makar,nhl,2.5,-182.0,135.0,-182.0,over,buffer


# Pinnacle NBA Scraping

In [63]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NBA/nba_matchups.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
player_props = []
#test = []
for i, d in enumerate(data['data']):
    try:
        if d['special']['category'].lower() == 'player props':
            data = {}
            data['prop_name'] = d['special']['description']
            data['prop_id'] = d['id']
            data['over_id'] = d['participants'][0]['id']
            data['under_id'] = d['participants'][1]['id']
            player_props.append(data)        
                
            
    except:
        continue

# Closing file
f.close()
#print(count)

In [64]:
# Opening JSON file
f = open(main_dir + "/Pinnacle/NBA/nba_straight.json")
  
# returns JSON object as 
# a dictionary
data = json.load(f)
prop_prices = []
for d in data['data']:
    try:

        row = {}
        row['prop_id'] = d['matchupId']
    
        row['over_id'] = d['prices'][0]['participantId']
        row['over_points'] = d['prices'][0]['points']
        row['over_price'] = d['prices'][0]['price']
    
        row['under_id'] = d['prices'][1]['participantId']
        row['under_points'] = d['prices'][1]['points']
        row['under_price'] = d['prices'][1]['price']   
 
        prop_prices.append(row)
    
    except:
        continue


    
# Closing file
f.close()
#print(count)

In [65]:
nba_propd = pd.DataFrame(player_props)
nba_prices = pd.DataFrame(prop_prices)

In [66]:
nba_df = pd.merge(nba_propd, nba_prices, how='left', on=['prop_id', 'over_id', 'under_id'])
nba_df.dropna(inplace=True)
nba_df['min_price'] = np.where(nba_df['over_price']<=nba_df['under_price'], 
                               nba_df['over_price'], nba_df['under_price'])

In [67]:
nba_df['player_name'] = nba_df['prop_name'].apply(player_name)
nba_df['prop_type'] = nba_df['prop_name'].apply(prop_type)

nba_df.drop(columns='prop_name', inplace=True)
nba_df.drop_duplicates(inplace=True)

In [68]:
nba = get_picks(fdf, nba_df, 'nba')
nba.sort_values(by='min_price', ascending=True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
480,6.0,assists,josh giddey,nba,5.5,-182.0,135.0,-182.0,over,buffer
513,3.5,rebounds,jaden mcdaniels,nba,3.5,-179.0,133.0,-179.0,over,equal
74,5.0,assists,cole anthony,nba,5.5,134.0,-177.0,-177.0,under,buffer
260,4.0,assists,ayo dosunmu,nba,3.5,-173.0,129.0,-173.0,over,buffer
486,14.0,points,josh giddey,nba,14.5,122.0,-160.0,-160.0,under,buffer
...,...,...,...,...,...,...,...,...,...,...
494,31.5,pts+rebs+asts,anthony edwards,nba,31.5,-116.0,-114.0,-116.0,over,equal
404,15.5,points,de'andre hunter,nba,15.5,-113.0,-116.0,-116.0,under,equal
377,21.5,pts+rebs+asts,clint capela,nba,21.5,-114.0,-116.0,-116.0,under,equal
485,7.5,rebounds,josh giddey,nba,7.5,-115.0,-115.0,-115.0,over,equal


# All Sports Dataframe

In [69]:
full_df = nhl.append(nfl).append(mlb)
pd.set_option('display.max_rows',None)
full_df.sort_values(by = 'min_price', ascending = True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
9,2.0,shots on goal,tony deangelo,nhl,2.5,146.0,-202.0,-202.0,under,buffer
18,1.0,points,nathan mackinnon,nhl,1.5,141.0,-195.0,-195.0,under,buffer
24,3.0,shots on goal,mikko rantanen,nhl,2.5,-192.0,141.0,-192.0,over,buffer
7,2.0,shots on goal,ivan provorov,nhl,2.5,140.0,-191.0,-191.0,under,buffer
30,3.0,shots on goal,kyle connor,nhl,2.5,-190.0,139.0,-190.0,over,buffer
10,4.0,hits allowed,blake snell,mlb,4.5,135.0,-185.0,-185.0,under,buffer
36,2.0,shots on goal,ryan o'reilly,nhl,1.5,-184.0,135.0,-184.0,over,buffer
25,3.0,shots on goal,valeri nichushkin,nhl,2.5,-184.0,135.0,-184.0,over,buffer
28,2.0,shots on goal,blake wheeler,nhl,2.5,135.0,-184.0,-184.0,under,buffer
4,3.5,shots on goal,matthew tkachuk,nhl,2.5,-183.0,134.0,-183.0,over,buffer


# Dropping Buffered Lines

In [70]:
equal_lines_df = full_df.loc[full_df['edge']!='buffer']
equal_lines_df.sort_values(by = 'min_price', ascending = True)

,pp_line,prop_type,player_name,league,pinnacle_line,over_price,under_price,min_price,favored,edge
52,0.5,total bases,jose trevino,mlb,0.5,-157.0,117.0,-157.0,over,equal
47,0.5,total bases,trey mancini,mlb,0.5,-144.0,108.0,-144.0,over,equal
22,0.5,total bases,bryson stott,mlb,0.5,-143.0,107.0,-143.0,over,equal
21,0.5,total bases,ha-seong kim,mlb,0.5,-142.0,107.0,-142.0,over,equal
35,2.5,shots on goal,jordan kyrou,nhl,2.5,107.0,-142.0,-142.0,under,equal
50,0.5,total bases,josh donaldson,mlb,0.5,-140.0,105.0,-140.0,over,equal
40,0.5,total bases,anthony rizzo,mlb,0.5,-139.0,105.0,-139.0,over,equal
45,0.5,total bases,giancarlo stanton,mlb,0.5,-139.0,104.0,-139.0,over,equal
14,0.5,points,sam bennett,nhl,0.5,-139.0,104.0,-139.0,over,equal
17,0.5,total bases,austin nola,mlb,0.5,-138.0,104.0,-138.0,over,equal
